# Generate Choice file for all participants (pet experiment)


In [9]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
# Useful to quickly extract csv files
import glob
import os

In [10]:
# Extract Rate data

In [11]:
# Load data all participants

rateNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/*_RateData_PFT.csv')
choiceNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/*_ChoiceData_PFT_MRI.csv')

rateNameFrames.sort()
choiceNameFrames.sort()

In [12]:
rateNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S1_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S2_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S3_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S4_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S5_RateData_PFT.csv']

In [13]:
choiceNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot1_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot2_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot3_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot4_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot5_ChoiceData_PFT_MRI.csv']

## Join rating in this case

In [14]:
partic_num=len(rateNameFrames) #number of participants

choiceFrameAll = pd.DataFrame( columns = ['trial','leftId','rightId','frame','endTime','choice','choiceTime','LRat1','RRat1','LRat2','RRat2','LVar1','RVar1','LVar2','RVar2','Part'])
idFrameAll = pd.DataFrame( columns = ['nameId','rat1','rat2','var1','var2','Part'])

for j in range (partic_num):
    rateFrame = pd.read_csv(rateNameFrames[j])
    choiceFrames = pd.read_csv(choiceNameFrames[j])
    
    if 'ratL' in choiceFrames.columns:
        if 'ratR' in choiceFrames.columns:
            choiceFrames = choiceFrames.drop(['ratL', 'ratR'], axis=1)

    
    idNames = list(rateFrame.picId.unique())
    IdRatings1 = [None]*len(idNames)
    IdRatings2 = [None]*len(idNames)
    IdRatingsVar1 = [None]*len(idNames)
    IdRatingsVar2 = [None]*len(idNames)

    
    #extract average rating for like, dislike
    for i in range(len(idNames)):
        IdRatings1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.mean()
        IdRatings2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.mean()
        
        IdRatingsVar1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.std()
        IdRatingsVar2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.std()

    rateValues = pd.DataFrame( {'nameId': idNames,'rat1': IdRatings1,'rat2': IdRatings2,'var1': IdRatingsVar1,'var2': IdRatingsVar2,})
    rateValues['Part'] = [j]*len(rateValues)    
    
    # save information about celebs and ratings
    
    idFrameAll = pd.concat([idFrameAll,rateValues],ignore_index=True)

    # separate ratings and rating-variability for each frame
    ChoiceLRat1 = [None]*len(choiceFrames)
    ChoiceRRat1 = [None]*len(choiceFrames)
    ChoiceLRat2 = [None]*len(choiceFrames)
    ChoiceRRat2 = [None]*len(choiceFrames)

    ChoiceLVar1 = [None]*len(choiceFrames)
    ChoiceRVar1 = [None]*len(choiceFrames)
    ChoiceLVar2 = [None]*len(choiceFrames)
    ChoiceRVar2 = [None]*len(choiceFrames)
    
    for i in range(len(choiceFrames)):
        
        choiceL = choiceFrames.iloc[i].leftId
        choiceR = choiceFrames.iloc[i].rightId
        
        if any((rateValues['nameId'] == choiceL)) and any((rateValues['nameId'] == choiceR)):
        
            ChoiceLRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat1.values[0]
            ChoiceRRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat1.values[0]
            ChoiceLRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat2.values[0]
            ChoiceRRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat2.values[0]
            
            ChoiceLVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var1.values[0]
            ChoiceRVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var1.values[0]
            ChoiceLVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var2.values[0]
            ChoiceRVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var2.values[0]

    choiceFrames['LRat1'] = ChoiceLRat1
    choiceFrames['RRat1'] = ChoiceRRat1
    choiceFrames['LRat2'] = ChoiceLRat2
    choiceFrames['RRat2'] = ChoiceRRat2 
    
    choiceFrames['LVar1'] = ChoiceLVar1
    choiceFrames['RVar1'] = ChoiceRVar1
    choiceFrames['LVar2'] = ChoiceLVar2
    choiceFrames['RVar2'] = ChoiceRVar2
     
    choiceFrames['Part'] = [j]*len(choiceFrames)    

    choiceFrameAll = pd.concat([choiceFrameAll,choiceFrames],ignore_index=True)
    

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Save file

In [15]:
choiceFrameAll

,LRat1,LRat2,LVar1,LVar2,Part,RRat1,RRat2,RVar1,RVar2,choice,...,primerSound,rightId,trial,trialEnd,trialOnset,trialOnsetTim,trialRespTime,trialSnd1Tim,trialSnd2Tim,trialThinkTime
0,1.0,4.0,NaN,NaN,0,3.0,2.0,NaN,NaN,1.0,...,1.0,2.0,0.0,14.011541,29.976978,0.016906,6.511024,1.065794,1.566313,3.510177
1,3.0,2.0,NaN,NaN,0,2.0,3.0,NaN,NaN,-1.0,...,1.0,6.0,1.0,16.016302,43.990964,0.014470,7.515057,1.053664,1.553902,3.514876
2,1.0,4.0,NaN,NaN,0,3.0,2.0,NaN,NaN,-1.0,...,1.0,2.0,2.0,18.015984,60.010107,0.057480,8.514907,1.102538,1.603700,3.514081
3,2.0,3.0,NaN,NaN,0,3.0,2.0,NaN,NaN,-1.0,...,2.0,2.0,3.0,16.000103,78.028215,0.014218,7.515609,1.053275,1.554410,3.514568
4,1.0,4.0,NaN,NaN,0,4.0,1.0,NaN,NaN,1.0,...,2.0,1.0,4.0,14.016120,94.030169,0.013254,6.515891,1.053375,1.554291,3.514665
5,1.0,4.0,NaN,NaN,0,2.0,3.0,NaN,NaN,-1.0,...,2.0,6.0,5.0,18.005992,108.075594,0.015995,8.504803,1.059895,1.559993,3.503521
6,4.0,1.0,NaN,NaN,0,2.0,3.0,NaN,NaN,1.0,...,2.0,6.0,6.0,14.000477,126.083210,0.013754,6.515656,1.054865,1.555044,3.515786
7,2.0,3.0,NaN,NaN,0,4.0,1.0,NaN,NaN,1.0,...,1.0,1.0,7.0,18.017106,140.085368,0.014195,8.515327,1.054390,1.554485,3.515110
8,3.0,2.0,NaN,NaN,0,2.0,3.0,NaN,NaN,1.0,...,1.0,6.0,8.0,14.015799,158.104346,0.014204,6.515508,1.054082,1.554342,3.514846
9,2.0,3.0,NaN,NaN,0,1.0,4.0,NaN,NaN,1.0,...,1.0,5.0,9.0,18.000865,172.121744,0.013727,8.500684,1.055417,1.555560,3.515337


In [16]:
choiceFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/All/PFT_Choice_Pilot_All.csv',index = False)
idFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/All/PFT_idRatingsFrame_Pilot_All.csv',index = False)